In [5]:

import io
import os
import shutil
import gzip

import pandas as pd


def decompress_gz(gz_path, output_path, remove_gz=True, fill_blank_gz=False):
    with gzip.open(gz_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"{gz_path}が解凍され、{output_path}に保存されました。")
    if remove_gz:
        os.remove(gz_path)

    if fill_blank_gz:
        with open(gz_path, 'w') as f:
            f.write("")


file="crawl-data_CC-MAIN-2023-50_segments_1700679099514.72_warc_CC-MAIN-20231128115347-20231128145347-00000.warc.gz"
decompress_gz(file, "test.warc", remove_gz=False, fill_blank_gz=True)


EOFError: Compressed file ended before the end-of-stream marker was reached

Error converting Pandas column with name: "id" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray


ArrowTypeError: Error converting Pandas column with name: "id" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray

In [49]:
df = pd.read_gbq(
    "SELECT * FROM `hatakeyamallm.cc_dataset.warcs` LIMIT 100",
    dialect="standard",
)
df

GenericGBQException: Reason: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/hatakeyamallm/queries?prettyPrint=false: Not found: Table hatakeyamallm:cc_dataset.warcs was not found in location US

In [37]:
from main import get_cc_path_list

batch_id = 1
n_batch = 3

start_idx, end_idx = (batch_id * n_batch,
                          (batch_id + 1) * n_batch)
cc_path_list = get_cc_path_list()
target_path_list = cc_path_list[start_idx:end_idx]
target_path_list

data/path_list/CC-MAIN-2023-50


['crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00003.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00004.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00005.warc.gz']

In [46]:
import glob


def get_cc_path_list(path_dir="data/path_list/*"):
    path_list = []
    for file_path in glob.glob(path_dir):
        print(file_path)
        with open(file_path, "r") as f:
            temp_path_list = f.readlines()

        temp_path_list = [path.strip() for path in temp_path_list]

        path_list += temp_path_list

    return path_list
path_list = get_cc_path_list()



df = pd.DataFrame(path_list)
df.to_csv("data/path_list.csv", index=False)

data/path_list/CC-MAIN-2023-50


In [54]:
import gzip
from warcio import ArchiveIterator

# .warc.gz ファイルを開く
with gzip.open('data.warc.gz', 'rb') as gz_file:
    # WARC レコードのストリーミング解析を開始
    for record in ArchiveIterator(gz_file):
        try:
            if record.rec_type == 'response':
                # レスポンス本文を取得
                payload = record.content_stream().read()
                # 必要に応じて処理を行う
                print(f"URL: {record.rec_headers.get_header('WARC-Target-URI')}")
                print(f"Content: {payload[:100]}")
        except Exception as e:
            print(e)
            continue
    


URL: http://0355car.com/info/1628649/131%E6%9C%9F%EF%BC%9A%E3%80%90%E5%8F%97%E7%9B%8A%E5%8C%AA%E6%B5%85%E3%80%91%E6%97%A0%E9%94%99%E7%BB%9D%E8%82%96
Content: b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="utf-8">\n\t<meta name="viewport" content="widt'
URL: http://03prk.ru/index.php?option=com_content&view=article&id=414:2023-06-05-08-54-31&catid=82:2018-10-09-09-20-02&Itemid=237
Content: b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtm'
URL: http://060608.it/it/accoglienza/dormire/case-e-appartamenti-per-vacanze/page/280
Content: b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtm'
URL: http://074f.blogbaiaodedois.com/preferred/preferred-thermal-paper
Content: b'\n<!doctype html>\n<html>\n<head>\n    <meta charset="utf-8">\n    <script language="javascript" src="/_/'
URL: http://080.222ing-ut.info/index.phtml?PUT=a_show&AID=189348&FID=267011
Content: b'<html><head><t

EOFError: Compressed file ended before the end-of-stream marker was reached

In [9]:
import requests
warc_path = "crawl-data/CC-MAIN-2023-50/segments/1700679518883.99/warc/CC-MAIN-20231211210408-20231212000408-00019.warc.gz"
def cc_path_to_urls(cc_path):
    base_url = "https://data.commoncrawl.org/"

    url = base_url + cc_path
    filename = cc_path.replace("/", "_")
    gz_path = f"tmp/data/gz/{filename}"
    warc_path = f"tmp/data/warc/{filename}".replace(".gz", "")

    return url, gz_path, warc_path
def download_file(url, save_path):
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"ファイルが正常にダウンロードされました: {save_path}")
    else:
        print(response.raw)
        print(f"ファイルのダウンロードに失敗しました。ステータスコード: {response.status_code}")
url, gz_path, warc_path = cc_path_to_urls(
    warc_path
)
download_file(url,save_path='file.warc.gz')

ファイルが正常にダウンロードされました: file.warc.gz


In [4]:
from langdetect import detect
from main import pre_clean
from bs4 import BeautifulSoup
import gzip
from warcio import ArchiveIterator
import os
# .warc.gz ファイルを開く
all_count = 0
meta_ja_count = 0

with gzip.open('file.warc.gz', 'rb') as gz_file:
    # WARC レコードのストリーミング解析を開始
    for record in ArchiveIterator(gz_file):
        if record.rec_type == 'response':
                    if record.http_headers.get_header('Content-Type') == 'text/html':
                        content = record.content_stream().read()

                        # PARSER_TYPE = os.environ.get("PARSER_TYPE", "html.parser")
                        PARSER_TYPE = "lxml"
                        if PARSER_TYPE == "lxml":
                            soup = BeautifulSoup(content, 'lxml')
                        elif PARSER_TYPE == "html":
                            soup = BeautifulSoup(content, 'html.parser')
                        else:
                            raise ValueError(
                                "PARSER_TYPE is not defined.please set environment PARSER_TYPE=lxml.parser or html.parser")
                        # <html>タグからlang属性を取得
                        html_tag = soup.find('html')
                        if html_tag and html_tag.has_attr('lang'):
                            lang = html_tag['lang']
                            texts = pre_clean(soup)
                            if soup.title is not None:
                                all_count += 1
                                title = soup.find('title').text
                                try:
                                    language = detect(str(title))
                                    if language == "ja":
                                        meta_ja_count += 1
                                except Exception as e:
                                    print(e)
                                    continue
                           
 
print(f"all_count: {all_count}")
print(f"meta_ja_count: {meta_ja_count}")
# print(f"ja_count: {ja_count}")

/var/folders/8n/nmxw7k9n74s82yxqjc2zfpk80000gn/T/ipykernel_32010/2925230499.py:21: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, 'lxml')


No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
all_count: 1099
meta_ja_count: 101


In [9]:
detect('福岡県北九州市八幡東区で安心安全な食材宅配サービスを利用するならOisix(オイシックス)がオススメ！')

'ja'

data/path_list/CC-MAIN-2023-50


['crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00076.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100452.79/warc/CC-MAIN-20231202203800-20231202233800-00144.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100427.59/warc/CC-MAIN-20231202140407-20231202170407-00210.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100452.79/warc/CC-MAIN-20231202203800-20231202233800-00180.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100427.59/warc/CC-MAIN-20231202140407-20231202170407-00600.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100327.70/warc/CC-MAIN-20231202042052-20231202072052-00657.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100448.65/warc/CC-MAIN-20231202172159-20231202202159-00090.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100308.37/warc/CC-MAIN-20231201215122-20231202005122-00741.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679100399.81/warc/CC-MAIN-20231202105028-20231

In [1]:
import glob
import pandas as pd

def get_cc_path_list(path_dir="data/rest_path_list/*"):
    path_list = []
    for file_path in glob.glob(path_dir):
        print(file_path)
        with open(file_path, "r") as f:
            temp_path_list = f.readlines()

        temp_path_list = [path.strip() for path in temp_path_list]

        path_list += temp_path_list

    return path_list
path_list = get_cc_path_list()

df = pd.DataFrame(path_list)
download_path_file = "data/download_path2.csv"
df_path = pd.read_csv(download_path_file, header=None)
df_path_list = df_path[0].tolist()

df_all = df[0].tolist()
df_rest_list = list(set(df_all) - set(df_path_list))
df_rest = pd.DataFrame(df_rest_list)
df_rest.to_csv("data/rest_path2.csv", index=False)

data/rest_path_list/rest_path.csv


In [8]:
import socket
# ホスト名を取得、表示
host = socket.gethostname()
print(host)

# ipアドレスを取得、表示
ip = socket.gethostbyname(host)
print(ip)

eijitnoMacBook-Pro-2.local
127.0.0.1


In [26]:
import trafilatura
from warcio.archiveiterator import ArchiveIterator
warc_url = 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/segments/1700679518883.99/warc/CC-MAIN-20231211210408-20231212000408-00019.warc.gz'
def contains_hiragana(text: str) -> bool:
        return any("\u3040" <= char <= "\u309F" for char in text)



resp = requests.get(warc_url, stream=True)
contents = []
urls = []
dates = []
all_count = 0 
for record in ArchiveIterator(resp.raw, arc2warc=True):
    all_count += 1
    content = record.content_stream().read()
    url= record.rec_headers.get_header("WARC-Target-URI")
    if url is None:
        continue
    
    if url.endswith(".pdf") or url.endswith(".jpg") or url.endswith(".png") or url.endswith(".jpeg"):
        continue
    if contains_hiragana(content.decode("utf-8", errors="ignore")):
        extracted_text = trafilatura.extract(content.decode("utf-8", errors="ignore"), include_formatting=True)
        if extracted_text is not None and contains_hiragana(extracted_text):
            urls.append(url)
            contents.append(extracted_text)
        dates.append(record.rec_headers.get_header("WARC-Date")) 

# 日本語の数
print(len(contents))
print(all_count)

KeyboardInterrupt: 

In [1]:
from main import extract_japanese_from_warc

ja_soup_list=extract_japanese_from_warc(path='file.warc.gz')

135it [00:00, 1251.80it/s]/Users/eijit/work/geniac_collect_jp/main.py:150: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, 'lxml')
111607it [01:12, 1539.38it/s]


In [3]:
ja_soup_list[0]

{'record_id': 23,
 'url': 'http://1173.jp/insp.html',
 'title': 'インスピレーション',
 'timestamp': '2023-12-11T23:14:02Z',
 'pre_cleaned_text': 'インスピレーションホームHome情報一覧Categoryメーカーを探す修理屋を探すお店を探す習い事を探す建設系を探す保険屋を探す老人ホームを探すアスリートを探す団体を探すサービスを探す掲載のご案内Informationよく頂く質問FaqリンクLinkお問い合わせContactホームHome情報一覧Category掲載のご案内Informationよく頂く質問FaqリンクLinkお問い合わせContact東京発ドメスティック サーフブランド INSPNEWおすすめTOKYOサーファー、格闘家、アーティスト、モトクロス、バスプロ、ダンサー、モデル など、自らのインスピレーションを信じ活躍する仲間と共に独自のブランドスタイルを発信している。INSP (インスピ）。INSPの語源はインスピーレーション公式サイトhttp://www.insp-web.net/社名株式会社フリーダムプラス住所〒171-0021 東京都豊島区西池袋4-38-8 荒井ビル1FTel/Fax03-5926-4474業務内容企画・生産・卸売販売・イベント・プロモーション他<< 前のページに戻る情報一覧メーカーを探す修理屋を探すお店を探す習い事を探す建設系を探す保険屋を探す老人ホームを探すアスリートを探す団体を探すサービスを探す不動産情報売買物件を探す賃貸物件を探すギャラリーいいなみ写真館管理者おすすめサイトRIKI HORIKOSHIスーパーアスリートnew日本サーフアカデミー高等部サーフィンに特化した通信制高校newＲＹＭＳスポーツマウスピース・マウスガード専門店newＸＪＭ販売促進newお知らせサーフボード修理屋さんサーフボード専門修理屋です。あらゆる難しい修理をスピーディに丁寧かつ綺麗に仕上げます。リペア伊藤建設ユニバーサルデザイン\u3000建築総合リフォーム\u3000伊藤建設株式会社リフォーム㈱マエダ・UD・サポート保険のことなら保険ステッカーの事なら斬字屋小さな物・量でも、良品質・低価格にこだわる看板メニュー掲載のご案